In [24]:
import re
import time
import json
import csv
import time
import nltk
import string
import numpy as np
import pandas as pd
from nltk import stem
from random import randint
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [86]:
#ratings_dataFrame = pd.read_csv("../Preprocess/ratings_dataFrame.csv", index_col=0)

books_dataFrame = pd.read_csv("../../Preprocess/books_dataFrame.csv", index_col=0)
books_dataFrame_new = pd.read_csv("../../Preprocess/books_dataFrame_new.csv", index_col=0)

#users_dataFrame = pd.read_csv("../Preprocess/users_dataFrame.csv", index_col=0, low_memory=False)


with open('../userDict.json', 'r') as fp:
    
    userDict = json.load(fp)
    
with open('../isbnDict.json', 'r') as fp:
    
    isbnDict = json.load(fp)
    
    
with open('featuresDict_new.json', 'r') as fp:
    
    featuresDict = json.load(fp)
    
    
with open('booksDict_new.json', 'r') as fp:
    
    booksDict = json.load(fp)
    
with open('usersDict_new.json', 'r') as fp:
    
    usersDict = json.load(fp)
    
with open('inverted_index.json', 'r') as fp:
    
    inverted_index = json.load(fp)
    
print("Ok")

Ok


In [26]:
def featuresExtraction(books_dataFrame_new):
    
    features = set()
     
    years = books_dataFrame_new["Year-Of-Publication"].values
    features.update(set(years))

    authors = books_dataFrame_new["Book-Author"].values
    features.update(set(authors))

    publishers = books_dataFrame_new["Publisher"].values
    features.update(set(publishers))
    
    titles = books_dataFrame_new["Book-Title"].values
    
    for title in titles:
        
        features.update(set(title.split()))
        
    featuresDict = {str(k):v for v,k in enumerate(features)}
    
    return featuresDict

In [150]:
featuresDict

{'autoritã': 2,
 'richard e leakey': 76086,
 'changewav': 109421,
 'entorn': 3,
 'william j bennett': 4,
 'hadrien': 5,
 'russell h cornwell': 6,
 'claddagh': 7,
 'james p lewis': 8,
 'instructional fair': 9,
 'queasi': 10,
 'jo voce': 59832,
 'klaus kowalski': 12,
 'jens christian grndahl': 13,
 'kristallwelt': 14,
 'robert st louis': 32685,
 'ribbentrop': 17,
 'j t richards': 164052,
 'theodoor hendrik van de velde': 100408,
 'robert j stevenson': 146171,
 'lundi': 20,
 'erfert fenton': 115654,
 'richards': 21,
 'signal hill publications': 22,
 'capyboppi': 139849,
 'luramedia': 24,
 'bruce andrews': 25,
 'duelo': 30,
 'informix press': 29,
 'fotografã': 32,
 'sharu': 86635,
 'jessica wilber': 33,
 'barnes amp noble inc ': 34,
 'william bell': 36,
 'tuckworth': 44,
 'damais': 38,
 'mccourtney': 41,
 'carol g wells': 39,
 'david j howe': 40,
 'andrã gide': 140597,
 'nutricion': 129051,
 'visick': 45,
 'distributed art publishers dap ': 46,
 'ancren': 96950,
 'andrea davis pinkney': 47

In [157]:
def invertedIndex(books_dataFrame_new, featuresDict):

    inverted_index = {k:{} for k in list(featuresDict.keys())}

    books = list(books_dataFrame_new.index)

    for book in books:

        for word in books_dataFrame_new["Book-Title"][book].split():

            inverted_index[word][book] = inverted_index[word].get(book,0) + 2.0

        year = str(books_dataFrame_new["Year-Of-Publication"][book])
        inverted_index[year][book] = inverted_index[year].get(book,0) + 0.1

        author = books_dataFrame_new["Book-Author"][book]
        inverted_index[author][book] = inverted_index[author].get(book,0) + 1.0

        publisher =  books_dataFrame_new["Publisher"][book]
        inverted_index[publisher][book] = inverted_index[publisher].get(book,0) + 0.3


    for word in inverted_index:

        tot = 0
        for book in inverted_index[word]:
            tot = tot + inverted_index[word][book]
        for book in inverted_index[word]:
            inverted_index[word][book] = inverted_index[word][book]/tot 

    tot_books = len(books)        
    tf_id = {k:0 for k in inverted_index}

    for word in tf_id:
        tf_id[word] = np.log(tot_books)/len(inverted_index[word])


    for word in inverted_index:
        for book in inverted_index[word]:
            inverted_index[word][book] = inverted_index[word][book]*tf_id[word]
            
    return inverted_index, tf_id

In [142]:
def Books(inverted_index, books_dataFrame_new):
    
    booksDict = {k:{} for k in list(books_dataFrame_new.index)}

    for word in inverted_index:

        for book in inverted_index[word]:

            booksDict[book][word] = inverted_index[word][book]
            
    return booksDict

In [138]:
def Users(booksDict,isbnDict):

    usersDict = {k:{} for k in userDict}


    for isbn in booksDict:

        try:
            for user in isbnDict[isbn]:

                t = int(isbnDict[isbn][user])
                if t > 5:

                    for feature in booksDict[isbn]:

                        usersDict[user][feature] =  usersDict[user].get(feature,0) + booksDict[isbn][feature] + t//5
        except:
            continue
            
        return usersDict

In [88]:
query = "Where You'll Find Me: And Other Stories"

In [151]:
def smallerInvertedIndex(inverted_index, t):
    v = []
    for word in inverted_index:
        for book in inverted_index[word]:
            if inverted_index[word][book] < t:

                v.append([word,book])
                
    for i in v:
        del inverted_index[i[0]][i[1]]
        
    return inverted_index

What do you prefer?

0 How Did We Find Out About Computers (Asimov, Isaac, How Did We Find Out--Series.)
1 Fantastic Beasts and Where to Find Them
2 The Mystery of the Secret Room (The 5 Find-outers)
3 Encyclopedia Brown Finds the Clues (Encyclopedia Brown (Paperback))
4 Finding Moon
3


In [220]:
def cleanQuery(query):

    stop = stopwords.words('english')
    stemmer = stem.PorterStemmer()
    wnl = WordNetLemmatizer()

    query = str(query).lower()
    query = re.sub(r"\W+" , " ", str(query)).split()    
    query = [wnl.lemmatize(word) for word in query if word not in stop]
    query = [stemmer.stem(word) for word in query]
    
    return query

def processQuery(q_list):
    
    books = set()
    i = 0
    for q in q_list:
        if i == 0:
            try:
                p = set(list(inverted_index[q].keys()))
                books = books.union(p)
            except:
                i = 1   
        else:
            try:
                p = set(inverted_index[q].keys())
                books = books.intersection(p) 
            except:
                continue        
    return books

def SimilarityinvertedIndex(isbnDict, featuresDict,booksDict, q_list, books, books_dataFrame):
    
    books = np.array(list(books))
    
    if len(books)>500:
        
        books = books[np.random.randint(0,len(books),500)]
    
    X_myQuery = np.zeros(len(featuresDict))

    # others users
    X_items = np.zeros((len(books),len(featuresDict)))


    for q in q_list:

        X_myQuery[featuresDict[q]] = 1

        
    for b in range(len(books)):

        try:
            for feature in booksDict[books[b]]:

                X_items[b][featuresDict[feature]] = booksDict[books[b]][feature]

        except:
            continue

    num = (X_myQuery*X_items).sum(axis=1)

    d_myQuery = np.sqrt((X_myQuery**2).sum())
    d_items = np.sqrt((X_items**2).sum(axis=1))

    res = num/(d_myQuery*d_items)
   
    ff = sorted(zip(books,res), key=lambda tup: tup[1], reverse=True)
    
    v = []
    titles = set()
    for f in ff[:20]:
        try:
            title = books_dataFrame["Book-Title"][f[0]]
            if title not in titles: 
                v.append([title,f[0]])

            titles.update([title])
        except:
            continue
    v = {k:v for k,v in enumerate(v)}
    
    return v

def mainSimilarity(isbnDict, featuresDict,booksDict,books_dataFrame):
    
    print("Search book:\n")
    query = input()

    q_list = cleanQuery(query)
    books = processQuery(q_list)
    ff = SimilarityinvertedIndex(isbnDict, featuresDict,booksDict, q_list, books, books_dataFrame)
    print("What do you prefer?\n")
    r = -1
    while r not in list(ff.keys()):
        for k in ff:

            print(k, ff[k][0])

        r = int(input())

    book_number = ff[r][1]
    
    return book_number 


def mainSimilarity(isbnDict, featuresDict,booksDict,books_dataFrame):
    
    print("Search book:\n")
    query = input()

    q_list = cleanQuery(query)
    books = processQuery(q_list)
    ff = SimilarityinvertedIndex(isbnDict, featuresDict,booksDict, q_list, books, books_dataFrame)
    print("What do you prefer?\n")
    r = -1
    while r not in list(ff.keys()):
        for k in ff:

            print(k, ff[k][0])

        r = int(input())

    book_number = ff[r][1]
    
    return book_number 